# Edge Detection Confusion Matrix - Optimization Ver.2
* learn 함수 최적화
    * 패턴 검색 방식 변경 : 
    for문 512번을 통해 검색하는 방식
    -> 검색한 후, 검색된 횟수에 따라 learn을 진행하는 방식
    * 정렬 방식 변경 : 
    learn마다 bubble sort 진행
    -> 정렬 함수 생성 후, 추론 직전 최종 정렬 함수 실행

# 이미지 전처리


In [1]:
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import time, inspect, os
import knockknock
from knockknock import telegram_sender

#import cv2

2022-11-16 17:13:15.401286: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/neurochip3/.pyenv/versions/3.9.8/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/neurochip3/.pyenv/versions/3.9.8/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/neurochip3/.pyenv/versions/3.9.8/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, wo

## 이미지 download / resize

In [2]:

(train_images, train_labels),(test_images, test_labels)= keras.datasets.mnist.load_data() 

image_size = 28
train_images = np.expand_dims(train_images, axis = -1)
train_images = tf.image.resize(train_images,[image_size,image_size])
train_images = np.squeeze(train_images)
train_images= train_images/255.0

test_images = np.expand_dims(test_images, axis = -1)
test_images = tf.image.resize(test_images,[image_size,image_size])
test_images = np.squeeze(test_images)
test_images= test_images/255.0
#이미지 resize


2022-11-16 17:13:18.304906: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-16 17:13:18.304933: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (clr-064ffa6064d8402181f1442d7a7d4225): /proc/driver/nvidia/version does not exist
2022-11-16 17:13:18.305241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 이미지 양자화
0.3, 0.7 기준으로 (0 / 0.5 / 1)로 나누기

In [3]:
for k in range(train_images.shape[0]) :
  for i in range(image_size):
    for j in range(image_size):
      if train_images[k, i,j] <0.3:
        train_images[k, i, j] = 0
      elif train_images[k, i,j] <0.7:
        train_images[k, i,j] = 0.5
      else :
        train_images[k, i,j] = 1

for k in range(test_images.shape[0]) :
  for i in range(image_size):
    for j in range(image_size):
      if test_images[k, i,j] <0.3:
        test_images[k, i, j] = 0
      elif test_images[k, i,j] <0.7:
        test_images[k, i,j] = 0.5
      else :
        test_images[k, i,j] = 1
# 0 0.5 1 셋 중 하나의 값으로 바꿔줌

## 데이터 나누기

In [4]:
###################데이터 나누기 
train = [[],[],[],[],[],[],[],[],[],[]]
for k in range(train_images.shape[0]):
  if train_labels[k] ==0:
    train[0].append(train_images[k])
  elif train_labels[k] ==1:
    train[1].append(train_images[k])
  elif train_labels[k] ==2:
    train[2].append(train_images[k])
  elif train_labels[k] ==3:
    train[3].append(train_images[k])
  elif train_labels[k] ==4:
    train[4].append(train_images[k])
  elif train_labels[k] ==5:
    train[5].append(train_images[k])
  elif train_labels[k] ==6:
    train[6].append(train_images[k])
  elif train_labels[k] ==7:
    train[7].append(train_images[k])
  elif train_labels[k] ==8:
    train[8].append(train_images[k])
  elif train_labels[k] ==9:
    train[9].append(train_images[k])
  else:
    train_images[k] = train_images[k]

# Chip / Layer 선언

## Chip

In [10]:
class chip:
  def __init__(self, size, threshold1 = 0.9, threshold2 = 0.95):
    '''
    size = memory input size (input pattern size)
    threshold1 = don't care  역치 //  0.9
    threshold2 = 1 역치 // 0.95
    '''
    self.size = size
    self.threshold1 = threshold1
    self.threshold2 = threshold2
    self.cam = []
    self.counter = []
    for i in range(2**size):
      self.cam.append(i)
      self.counter.append(0)

  def reset(self):
    '''
    cam의 size를 입력받은 값으로 초기화
    cam의 element를 0으로 초기화
    '''
    self.cam = []
    self.counter = []
    for i in range(2**self.size):
      self.cam.append(i)
      self.counter.append(0)

  def pattern_to_number(self, pattern):
    '''
    입력받은 pattern을 number로 변환
    (learn / get address 함수 내에서만 사용되므로 외부에서 사용할 필요 없음)
    '''
    num = 0
    result = []
    for i in range(len(pattern)):
      if pattern[i] == 1:
        num = num + 2**(len(pattern)-1-i)

    result.append(num)
    for i in range(len(pattern)):
      if pattern[i]==0.5:
        for k in range(len(result)):
          result.append(result[k]+2**(len(pattern)-1-i))
    
    return result

  def change_threshold(self, threshold1, threshold2):
      self.threshold1 = threshold1
      self.threshold2 = threshold2

  def learn(self, pattern):
    '''
    입력받은 pattern(을 number로 변환한 값)이 cam[i]와 같으면(count!=0) counter 배열의 element를 1 더해주고,
    counter 배열의 element에 따라 counter 배열과 cam 배열의 element를 동시에 정렬해 준다
      (더 큰 count 횟수 -> 더 큰 index의 counter/cam 배열의 element)
    '''
    num = self.pattern_to_number(pattern)
    self.counter[self.cam.index(num[0])] += 1
    '''
    ### 효율화 이전 ###
    for i in range(len(self.cam)):
      if num.count(self.cam[i])!=0:
        self.counter[i] = self.counter[i] +1

    for i in range(len(num)):
      for j in range(len(self.cam)-1):
        if self.counter[j] > self.counter[j+1]:
          temp =self.counter[j]
          self.counter[j] = self.counter[j+1]
          self.counter[j+1] = temp
          temp =self.cam[j]
          self.cam[j] = self.cam[j+1]
          self.cam[j+1] = temp
    '''
  def sort(self):
      d = dict()
      for i in range(len(self.counter)):
          d[cam[i]] = counter[i]
      d = sorted(d.items(), key = lambda item: item[1])
      self.counter = []
      self.cam = []
      for i in range(len(d)):
          self.counter.append(d[i][1])
          self.cam.append(d[i][0])
          
    def get_address(self, pattern):
    '''
    입력받은 pattern에 대한 address(cam?의 index)를 반환
    '''
    num = self.pattern_to_number(pattern)
    result = []
    for i in range(len(num)):
      result.append(self.cam.index(num[i]))
    return result
  
  def get_binary(self, pattern):
    '''
    입력받은 pattern에 대한 출력값(0, 0.5, 1)을 반환
      (get_address의 result가
      하단 임계점을 넘으면 0.5, 상단 임계점을 넘으면 1 반환)
    '''
    result = self.get_address(pattern)
    address = max(result)
    percent = address/len(self.cam)

    if percent < self.threshold1 :
      return 0.0
    elif percent <self.threshold2:
      return 0.5
    else:
      return 1.0


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 86)

## Layer

In [ ]:
class layer:
    def __init__(self, window_size, layer_size, stride = 0, threshold1 = 0.9, threshold2 = 0.95):
        self.L = [[0]*layer_size for _ in range(layer_size)]
        self.threshold1 = threshold1
        self.threshold2 = threshold2
        for i in range(layer_size):
            for j in range(layer_size):
                self.L[i][j] = chip(window_size*window_size, self.threshold1, self.threshold2)
        
        if stride ==0:
            self.stride = window_size
        else :
            self.stride = stride
        self.window_size = window_size
        self.layer_size = layer_size

    def reset(self):
        '''
        layer가 초기화
        내부 chip의 배열도 초기화
        '''
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                self.L[i][j].reset()

    def change_threshold(self, threshold1, threshold2):
        self.threshold1 = threshold1
        self.threshold2 = threshold2
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                self.L[i][j].change_threshold(threshold1, threshold2)

    def learn(self, image):
        '''
        이미지 하나를 layer가 입력받게 되는 함수
        이미지 크기 : i * j
        window(pattern) 크기 : a * b
        '''
        # 학습 가능한 이미지 사이즈인지 확인하는 코드 추가 작성
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                pattern= []
                for a in range(self.window_size):
                    for b in range(self.window_size):
                        pattern.append(image[self.stride*i+a][self.stride*j+b])
                self.L[i][j].learn(pattern)
    
    def sort(self):
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                self.L[i][j].sort()
    
    def out(self, image):
        '''
        이미지 넣고 출력되는 layer out 보여줌
        '''
        # 입력 가능한 이미지 사이즈인지 확인하는 코드 추가 작성
        out_image = [[0] * self.layer_size for _ in range(self.layer_size)]
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                pattern=[]
                for a in range(self.window_size):
                    for b in range(self.window_size):
                        pattern.append(image[self.stride*i +a][self.stride*j +b])
                out_image[i][j] = self.L[i][j].get_binary(pattern)
        return out_image

    def out_address(self,image):
        '''
        마지막 레이어의 주소를 반환
        '''
        out_image = [[0] * self.layer_size for _ in range(self.layer_size)]
        for i in range(self.layer_size):
            for j in range(self.layer_size):
                pattern=[]
                for a in range(self.window_size):
                    for b in range(self.window_size):
                        pattern.append(image[self.stride*i +a][self.stride*j +b])
                out_image[i][j] = max(self.L[i][j].get_address(pattern))
        return out_image   

    def out_size(self):
        '''
        N*N 출력 이미지에서 N(size)이 나옴 
        '''
        return self.layer_size

# Edge Detection

## 6 - Filtering CAM(`Chip`) 학습
size 4의 Chip을 선언하고 수평, 수직, 반대각, 주대각을 학습하는 데이터를 학습한다.


In [ ]:
edge_detection = chip(4,threshold1 = 14/16, threshold2=14/16)

@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def filtering_cam_learning():
    #수평 = H
    for i in range (8):
        edge_detection.learn([1,1,0,0])
        edge_detection.learn([0,0,1,1])
    #수직 = V
    for i in range (6):
        edge_detection.learn([1,0,1,0])
        edge_detection.learn([0,1,0,1])
    #반대각(ㅢ) = R
    for i in range (4):
        edge_detection.learn([1,0,0,0])
        edge_detection.learn([0,1,1,1])
    #주대각(ㄴ) = L
    for i in range (2):
        edge_detection.learn([0,1,0,0])
        edge_detection.learn([1,0,1,1])

filtering_cam_learning()

## 7 - Edge Detection : `Chip` 배열
H, V, R, L의 배열(0 - 9)을 선언하고 각 배열의 element를 27*27로 만들어 준다.

전처리한 image(`data`)를 입력해 수평/수직/반대각/주대각(H, V, R, L)의 배열에 해당 패턴이 발견되면 1, 발견되지 않으면 0으로 입력한다.

`k`는 0 - 9

`a`는 각 숫자의 mnist 이미지

`i`, `j`는 이미지의 위치


In [ ]:
H = [[],[],[],[],[],[],[],[],[],[]]
V = [[],[],[],[],[],[],[],[],[],[]]
R = [[],[],[],[],[],[],[],[],[],[]]
L = [[],[],[],[],[],[],[],[],[],[]]

@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_edge_detection():
    for k in range(10):
        for a in range(len(train[k])) :
            data = train[k][a]    
            H[k].append([[0] * 27 for _ in range(27)])
            V[k].append([[0] * 27 for _ in range(27)])
            R[k].append([[0] * 27 for _ in range(27)])
            L[k].append([[0] * 27 for _ in range(27)])
            for i in range(27):
                for j in range(27):
                    address = []
                    pattern = []
                    pattern = [(data[i][j]),(data[i][j+1]),
                            (data[i+1][j]),(data[i+1][j+1])]
                    address = edge_detection.get_address(pattern)
                    if 15 in address:
                        H[k][a][i][j] = 1
                    elif 14 in address:
                        H[k][a][i][j] = 1
                    else : H[k][a][i][j] = 0

                    if 13 in address:
                        V[k][a][i][j] = 1
                    elif 12 in address:
                        V[k][a][i][j] = 1
                    else : V[k][a][i][j] = 0

                    if 11 in address:
                        R[k][a][i][j] = 1
                    elif 10 in address:
                        R[k][a][i][j] = 1
                    else : R[k][a][i][j] = 0

                    if 9 in address:
                        L[k][a][i][j] = 1
                    elif 8 in address:
                        L[k][a][i][j] = 1
                    else : L[k][a][i][j] = 0

modellearning_edge_detection()

In [ ]:
Layer_H1 = [0,0,0,0,0,0,0,0,0,0]
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_layer_H():
    for k in range(10):
        Layer_H1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in H[k]:
            Layer_H1[k].learn(data)
Layer_V1 = [0,0,0,0,0,0,0,0,0,0]
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_layer_V():
    for k in range(10):
        Layer_V1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in V[k]:
            Layer_V1[k].learn(data)
Layer_R1 = [0,0,0,0,0,0,0,0,0,0]
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_layer_R():
    for k in range(10):
        Layer_R1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in R[k]:
            Layer_R1[k].learn(data)

Layer_L1 = [0,0,0,0,0,0,0,0,0,0]
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_layer_L():
    for k in range(10):
        Layer_L1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in L[k]:
            Layer_L1[k].learn(data)


In [ ]:
Layer_H1 = [0,0,0,0,0,0,0,0,0,0]
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def multi_modellearning_layer_H():
    for k in range(10):
        Layer_H1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in H[k]:
            Layer_H1[k].learn(data)
Layer_V1 = [0,0,0,0,0,0,0,0,0,0]
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def multi_modellearning_layer_V():
    for k in range(10):
        Layer_V1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in V[k]:
            Layer_V1[k].learn(data)
Layer_R1 = [0,0,0,0,0,0,0,0,0,0]
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def multi_modellearning_layer_R():
    for k in range(10):
        Layer_R1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in R[k]:
            Layer_R1[k].learn(data)

Layer_L1 = [0,0,0,0,0,0,0,0,0,0]
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def multi_modellearning_layer_L():
    for k in range(10):
        Layer_L1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in L[k]:
            Layer_L1[k].learn(data)


In [ ]:
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def now_not_multi_threding():
    multithreding = "not_multi-thrhreding"
    return multithreding
now_not_multi_threding()

Layer_H1 = [0,0,0,0,0,0,0,0,0,0]
modellearning_layer_H()
Layer_V1 = [0,0,0,0,0,0,0,0,0,0]
modellearning_layer_V()
Layer_R1 = [0,0,0,0,0,0,0,0,0,0]
modellearning_layer_R()
Layer_L1 = [0,0,0,0,0,0,0,0,0,0]
modellearning_layer_L()

In [ ]:
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def now_multi_PROCESSING():
    multi = "multi-PROCESSING"
    return multi
now_multi_PROCESSING()

In [ ]:
import multiprocessing

p1 = multiprocessing.Process(name = "subprocess1", target=multi_modellearning_layer_H)
p2 = multiprocessing.Process(name = "subprocess2", target=multi_modellearning_layer_V)
p3 = multiprocessing.Process(name = "subprocess3", target=multi_modellearning_layer_R)
p4 = multiprocessing.Process(name = "subprocess4", target=multi_modellearning_layer_L)

p1.start()
p2.start()
p3.start()
p4.start()

In [ ]:
p1.join()
p2.join()
p3.join()
p4.join()

In [ ]:
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def now_multi_threding():
    multithreding = "multi-thrhreding"
    return multithreding
now_multi_threding()

In [ ]:
import threading
import time
Layer_H1 = [0,0,0,0,0,0,0,0,0,0]
Layer_V1 = [0,0,0,0,0,0,0,0,0,0]
Layer_R1 = [0,0,0,0,0,0,0,0,0,0]
Layer_L1 = [0,0,0,0,0,0,0,0,0,0]

#if __name__ == "__main__":
t1 = threading.Thread(target=multi_modellearning_layer_H)
t2 = threading.Thread(target=multi_modellearning_layer_V)
t3 = threading.Thread(target=multi_modellearning_layer_R)
t4 = threading.Thread(target=multi_modellearning_layer_L)

t1.start()
t2.start()
t3.start()
t4.start()

In [ ]:
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def now_not_multi_threding():
    multithreding = "not_multi-thrhreding"
    return multithreding
now_not_multi_threding().py

In [ ]:
Layer_H1 = [0,0,0,0,0,0,0,0,0,0]
modellearning_layer_H()
Layer_V1 = [0,0,0,0,0,0,0,0,0,0]
modellearning_layer_V()

# Memory Learning **⟵ 여기부터**

## 8 - Layer 학습

Edge detection을 완료한 image(`data in H[k] - L[k]`)를 Layer_H1[0] - Layer_H1[9], Layer_V1[0 - 9], Layer_R1[0 - 9], Layer_L1[0 - 9]에 대해 학습(`learn`)

`window size`=3
`layer size`=9

### 함수 정의

In [ ]:
# 일반 함수
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_layer_H():
    for k in range(10):
        Layer_H1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in H[k]:
            Layer_H1[k].learn(data)
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_layer_V():
    for k in range(10):
        Layer_V1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in V[k]:
            Layer_V1[k].learn(data)
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_layer_R():
    for k in range(10):
        Layer_R1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in R[k]:
            Layer_R1[k].learn(data)
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_layer_L():
    for k in range(10):
        Layer_L1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in L[k]:
            Layer_L1[k].learn(data)

# multi 함수
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def optimizated_modellearning_layer_H():
    for k in range(10):
        Layer_H1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in H[k]:
            Layer_H1[k].learn(data)
    for k in range(10):
        Layer_H1[k].sort()
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def optimizated_modellearning_layer_V():
    for k in range(10):
        Layer_V1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in V[k]:
            Layer_V1[k].learn(data)
    for k in range(10):
        Layer_V1[k].sort()
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def optimizated_modellearning_layer_R():
    for k in range(10):
        Layer_R1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in R[k]:
            Layer_R1[k].learn(data)
    for k in range(10):
        Layer_R1[k].sort()
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def optimizated_modellearning_layer_L():
    for k in range(10):
        Layer_L1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in L[k]:
            Layer_L1[k].learn(data)
    for k in range(10):
        Layer_L1[k].sort()


### Not - multi - 

In [ ]:
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def now_not_multi_threding():
    multithreding = "not_multi-thrhreding"
    return multithreding
now_not_multi_threding()

Layer_H1 = [0,0,0,0,0,0,0,0,0,0]
modellearning_layer_H()
Layer_V1 = [0,0,0,0,0,0,0,0,0,0]
modellearning_layer_V()
Layer_R1 = [0,0,0,0,0,0,0,0,0,0]
modellearning_layer_R()
Layer_L1 = [0,0,0,0,0,0,0,0,0,0]
modellearning_layer_L()

### multi - processing

In [ ]:
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def now_multi_PROCESSING():
    multi = "multi-PROCESSING"
    return multi
now_multi_PROCESSING()

import multiprocessing

p1 = multiprocessing.Process(name = "subprocess1", target=multi_modellearning_layer_H)
p2 = multiprocessing.Process(name = "subprocess2", target=multi_modellearning_layer_V)
p3 = multiprocessing.Process(name = "subprocess3", target=multi_modellearning_layer_R)
p4 = multiprocessing.Process(name = "subprocess4", target=multi_modellearning_layer_L)

p1.start()
p2.start()
p3.start()
p4.start()

p1.join()
p2.join()
p3.join()
p4.join()

### multi - threading

In [ ]:
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def now_multi_threding():
    multithreding = "multi-thrhreding"
    return multithreding
now_multi_threding()

import threading
import time
Layer_H1 = [0,0,0,0,0,0,0,0,0,0]
Layer_V1 = [0,0,0,0,0,0,0,0,0,0]
Layer_R1 = [0,0,0,0,0,0,0,0,0,0]
Layer_L1 = [0,0,0,0,0,0,0,0,0,0]

#if __name__ == "__main__":
t1 = threading.Thread(target=multi_modellearning_layer_H)
t2 = threading.Thread(target=multi_modellearning_layer_V)
t3 = threading.Thread(target=multi_modellearning_layer_R)
t4 = threading.Thread(target=multi_modellearning_layer_L)

t1.start()
t2.start()
t3.start()
t4.start()

### 기존 방식(백업용)

In [ ]:
Layer_H1 = [0,0,0,0,0,0,0,0,0,0]
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_layer_H():
    for k in range(10):
        Layer_H1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in H[k]:
            Layer_H1[k].learn(data)
modellearning_layer_H()        

Layer_V1 = [0,0,0,0,0,0,0,0,0,0]
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_layer_V():
    for k in range(10):
        Layer_V1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in V[k]:
            Layer_V1[k].learn(data)
modellearning_layer_V()

Layer_R1 = [0,0,0,0,0,0,0,0,0,0]
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_layer_R():
    for k in range(10):
        Layer_R1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in R[k]:
            Layer_R1[k].learn(data)
modellearning_layer_R()

Layer_L1 = [0,0,0,0,0,0,0,0,0,0]
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modellearning_layer_L():
    for k in range(10):
        Layer_L1[k] = layer(3,9, stride= 0, threshold1 = 0.90, threshold2 = 0.96)
        for data in L[k]:
            Layer_L1[k].learn(data)
modellearning_layer_L()

# Model Validation

## 9 - Validation data Edge detection / Searching
위의 H, V, R, L과 동일한 방식으로 H_temp, V_temp, R_temp, L_temp 학습

i = 0부터 9까지 10번 반복, Layer_H1[0] - Layer_H1[9], Layer_V1[0 - 9], Layer_R1[0 - 9], Layer_L1[0 - 9]에 대해 반복 수행

H,V,R,L 중 가장 큰 값을 도출하는(maximum of (`H[0] + V[0] + R[0] + L[0]` - `H[9] + V[9] + R[9] + L[9]`)) 값을 `answer`로 출력.

정답과 같으면 `right = right + 1`, 틀리면 `wrong = wrong + 1`


In [ ]:
right = 0
wrong = 0
H_temp = [[0] * 27 for _ in range(27)]
V_temp = [[0] * 27 for _ in range(27)]
R_temp = [[0] * 27 for _ in range(27)]
L_temp = [[0] * 27 for _ in range(27)]
confusion = [[0] * 10 for _ in range(10)]

for k in range(len(test_images)):
    data = test_images[k]
    out_address = []
    out_result=[]

    for i in range(27):
        for j in range(27):
            pattern = []
            pattern = [(data[i][j]),(data[i][j+1]),
                      (data[i+1][j]),(data[i+1][j+1])]
            address = edge_detection.get_address(pattern)
            if 15 in address:
                H_temp[i][j] = 1
            elif 14 in address:
                H_temp[i][j] = 1
            else : H_temp[i][j] = 0

            if 13 in address:
                V_temp[i][j] = 1
            elif 12 in address:
                V_temp[i][j] = 1
            else : V_temp[i][j] = 0

            if 11 in address:
                R_temp[i][j] = 1
            elif 10 in address:
                R_temp[i][j] = 1
            else : R_temp[i][j] = 0
            
            if 9 in address:
                L_temp[i][j] = 1
            elif 8 in address:
                L_temp[i][j] = 1
            else : L_temp[i][j] = 0

#여기까지 H_temp, V_temp, R_temp, L_temp 초기화
#여기부터 i = 0부터 9까지 10번 반복, Layer_H1[0]부터 Layer_H1[9]까지, V, R, L까지 반복 수행

    for i in range(10):
        out_H = Layer_H1[i].out(H_temp)
        out_V = Layer_V1[i].out(V_temp)
        out_R = Layer_R1[i].out(R_temp)
        out_L = Layer_L1[i].out(L_temp)

        out_H_ = sum(out_H,[])
        out_V_ = sum(out_V,[])
        out_R_ = sum(out_R,[])
        out_L_ = sum(out_L,[])

        out = sum(out_H_)+sum(out_V_)+sum(out_R_)+sum(out_L_)
        out_result.append(out)
        
    answer = out_result.index(max(out_result))

    confusion[test_labels[k]][answer] = confusion[answer][test_labels[k]] + 1
    if answer == test_labels[k]:
        right = right +1
    else :
        wrong = wrong + 1
print(right / (right + wrong))

@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modelvalidation(input):
  print(input)
  return input
modelvalidation(right / (right + wrong))

## 결과 시각화(table)

In [ ]:
import pandas as pd
conf = [0,0,0,0,0,0,0,0,0,0]

for i in range(10):
    conf[i] = 100/sum(confusion[i])

for i in range(10):
    for j in range(10):
        confusion[i][j] = confusion[i][j]*conf[i]
    
df = pd.DataFrame(data = {'label_0' : confusion[0],
                          'label_1' : confusion[1],
                          'label_2' : confusion[2],
                          'label_3' : confusion[3], 
                          'label_4' : confusion[4],
                          'label_5' : confusion[5],
                          'label_6' : confusion[6],
                          'label_7' : confusion[7],
                          'label_8' : confusion[8],
                          'label_9' : confusion[9]
                         }
                 ,index = ['0','1','2','3','4','5','6','7','8','9']
                 )

df
@telegram_sender(token="5594017285:AAGaj_6-ZIx-QKXVixZb35X_ACcCVQr6w-U", chat_id=16579436)
def modelvalidation_calc_accuracy():
  overall_accuracy = ((confusion[0][0] + confusion[1][1] + confusion[2][2] + confusion[3][3] + confusion[4][4] + confusion[5][5] + confusion[6][6] + confusion[7][7] + confusion[8][8] + confusion[9][9]) / 10)
  #print((confusion[0][0] + confusion[1][1] + confusion[2][2] + confusion[3][3] + confusion[4][4] + confusion[5][5] + confusion[6][6] + confusion[7][7] + confusion[8][8] + confusion[9][9]))
  print(overall_accuracy)
  return overall_accuracy
modelvalidation_calc_accuracy()

currentfilename = inspect.getfile(inspect.currentframe()) #현재 파일이 >위치한 경로 + 현재 파일명
currentfilename = (currentfilename.split("\\")[-1]).split('.')[0] #현재 파일명만 추출
print(currentfilename, " : COMPLETED") #현재 파일명 : COMPLETED 출력
filename = time.strftime("result/%y%m%d_%H%M%S[" + currentfilename + "].txt", time.localtime(time.time())) #결과 파일명 정의
outputfile = open(filename, 'w') #결과 파일 생성
outputfile.write('COMPLETED AT : ' + time.strftime('%y-%m-%d %a_%H:%M:%S', time.localtime(time.time()))) #파일 : 완료된 시각
outputfile.write('\n' + "   RESULT OF : " + currentfilename + '\n') #파일 : 완료된 파일명
outputfile.write('\n' + df + '\n') #파일 : 완료된 파일명
outputfile.close #파일 편집 종료outputfile = open(filename, 'a') #결과 파일 생성

In [ ]:
#confusion.size()
df

In [ ]:
for i in range(10):
    for j in range(9):
        for k in range(9):
            if(Layer_H1[i].L[j][k].counter[384] != 0) :
                print("i: ", i, "j: ", j, "k: ", k)
                print(Layer_H1[i].L[j][k].cam)
                print(Layer_H1[i].L[j][k].counter) 